In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
cd /kaggle/input

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:


IMAGE_SIZE = [224, 224]

train_path = './tomatoleaf/tomato/train'
valid_path = './tomatoleaf/tomato/val'

In [ ]:

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in inception.layers:
  layer.trainable = False 

In [ ]:
 glob('./tomatoleaf/tomato/train/*')

In [ ]:
folders = glob('./tomatoleaf/tomato/train/*')
  

x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=prediction)

model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
cd /kaggle/input

In [ ]:
ls

In [ ]:
training_set = train_datagen.flow_from_directory('./tomatoleaf/tomato/train',
                                                 target_size = (224, 224),
                                                 batch_size = 15,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./tomatoleaf/tomato/val',
                                            target_size = (224, 224),
                                            batch_size = 15,
                                            class_mode = 'categorical')

print(training_set.classes)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

In [ ]:
len(test_set)

In [ ]:
# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=2,
  callbacks=[early_stopping],
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
predictions = model.predict(test_set)

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
y_pred = np.argmax(predictions, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(r, training_set, test_set)  
plt.show() 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# list all data in history
# measure accuracy
print(r.history.keys())
# summarize history for accuracy
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
cd /kaggle/working

In [ ]:

from tensorflow.keras.models import load_model

model.save('tomato_leaf_disease_detection.h5')